In this notebook, we'll look at building a visualiser to view the VPU applied en-mass to FFT data.

We can make our SensorSource objects iterables that return a next frame of data - https://www.programiz.com/python-programming/iterator ```__iter__``` just returns self (with any initialisation) and ```__next__``` returns self.read().

Our SensorSource objects also need a way of returning the size of the frame.

# Testing Sensor Sources

In [20]:
from src.sources.capture import VideoSource

video = VideoSource()
video.start()
grabbed, frame = video.read()
print(grabbed)
print(frame.shape)

video.stop()

True
(480, 640, 3)


In [21]:
from src.sources.capture import AudioSource
import time
import numpy as np

audio = AudioSource()
audio.start()
time.sleep(0.5)
# Test read
length1, samples1 = audio.read()
assert length1
assert samples1.any()
# Check starting and getting a frame
audio.start()
time.sleep(0.5)
length2, samples2 = audio.read()
assert length2
assert not np.array_equal(samples1, samples2)
print(samples2.shape)
# Test stopping
audio.stop()
assert not audio.started

[!] Asynchroneous capturing has already been started.
(65536,)


In [22]:
from src.sources.capture import CombinedSource, SensorSource

combined = CombinedSource()
type(combined.sources) == dict

True

In [23]:
len(combined.sources)

0

In [24]:
assert type(combined.sources) == dict
assert len(combined.sources) == 0
# Adding a source
combined.add_source(SensorSource())

In [25]:
list(combined.sources.items())[0][0]

'SensorSource'

In [26]:
from src.sources.capture import AVCapture

av = AVCapture()
av.start()
time.sleep(0.25)
data = av.read()
print(data)
av.stop()

{'audio': array([   0,    0,    0, ..., 2927, 2824, 3613], dtype=int16), 'video': array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 241, 184],
        [255, 241, 184],
        [255, 240, 183],
        ...,
        [130, 124, 155],
        [129, 123, 153],
        [127, 122, 152]],

       [[255, 240, 185],
        [255, 240, 185],
        [255, 240, 185],
        ...,
        [119, 132, 155],
        [122, 130, 161],
        [120, 128, 159]],

       [[255, 237, 188],
        [255, 237, 188],
        [255, 238, 186],
     

## Test FFT Source

In [79]:
from src.sources.fft import FFTSource

fft = FFTSource()
fft.start()

In [86]:
_, data = fft.read()
print(data)

[  0   0   0  33   0  42 110   0 185 162 245 172 128 223 157 206  81 130
 200  90 179 221 167 153 163 103 159 168 159 188 242 216 130  51 134 243
  16  45  77 250 234 199 198  58 211  99   1 212 162 232 189 156  36 209
 139  61  82 180 114 140 212 210 103  87  94 112 106  95  48 107 200 112
  50  76 142  71  78  43  57  72  80 116  79  52  36  32  58  50  42  21
  18  41  35  27  34  31  17  23  23  17  20  10   9   9   8   8   7   6
   6   6   6   6   5   5   5   5   5   5   5   5   5   4   5   4   4   4
   4   4   4   4   4   4   4   4   4   4   4]


In [78]:
fft.stop()

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


# Test Covariance Unit

In [27]:
from src.var_processor.covariance import CovarianceUnit
cov_unit = CovarianceUnit(2)
print(cov_unit.x_sum, cov_unit.square_sum, cov_unit.covariance, sep="\n", end="\n---\n")
assert not cov_unit.x_sum.any()
assert not cov_unit.square_sum.any()
# Test updating with data
ones = np.ones(shape=(2,1))
cov_unit.update(ones)
assert cov_unit.count == 1
assert np.array_equal(cov_unit.x_sum, ones)
assert np.array_equal(cov_unit.mean, ones)
assert not cov_unit.covariance.any()
print(cov_unit.x_sum, cov_unit.square_sum, cov_unit.covariance, sep="\n", end="\n---\n")
threes = ones*3
cov_unit.update(threes)
assert cov_unit.count == 2
assert np.array_equal(cov_unit.x_sum, ones+threes)
assert cov_unit.square_sum.any()
assert np.array_equal(cov_unit.mean, ones*2)
assert cov_unit.covariance.any()
print(cov_unit.x_sum, cov_unit.square_sum, cov_unit.covariance, sep="\n", end="\n---\n")

[[0.]
 [0.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
---
[[1.]
 [1.]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
---
[[4.]
 [4.]]
[[0.66666667 0.66666667]
 [0.66666667 0.66666667]]
[[0.33333333 0.33333333]
 [0.33333333 0.33333333]]
---


In [28]:
from src.var_processor.power_iterator import PowerIterator

power = PowerIterator(2)
power.ev

array([[0.83022113],
       [0.55743419]])

In [29]:
from src.var_processor.vpu import VPU

# Intialise VPU
vpu = VPU(2)
# Test Iteration
in_1 = np.random.randint(255, size=(2, 1))
in_1 = in_1 / in_1.max()
print(in_1)
r, residual = vpu.iterate(in_1)
print(r, residual)
r, residual = vpu.iterate(in_1)
print(r, residual)

[[0.3826087]
 [1.       ]]
[[0.80731506]] [[-0.33019738]
 [ 0.6209682 ]]
[[0.80731506]] [[-0.33019738]
 [ 0.6209682 ]]


In [30]:
for _ in range(0, 100):
    in_1 = np.random.randint(255, size=(2, 1))
    in_1 = in_1 / in_1.max()
    vpu.update_cov(in_1)
r, residual = vpu.iterate(in_1)
print(r, residual)

[[0.97765689]] [[0.02347528]
 [0.48736612]]


In [31]:
vpu.cu.covariance

array([[ 0.09543691, -0.06212411],
       [-0.06212411,  0.11117773]])

In [32]:
vpu.pi.eigenvector

array([[ 0.99884195],
       [-0.04811187]])

In [33]:
vpu.pi.cov

array([[ 0.09543691, -0.06212411],
       [-0.06212411,  0.11117773]])

In [34]:
self = vpu.pi
self.ev = np.matmul(np.power(self.cov, 1), self.ev)
# Scale to have unit length (convert to integer values?)
# self.ev = self.ev / np.linalg.norm(self.ev)
print(self.ev)

[[ 0.0983153 ]
 [-0.06740113]]


In [35]:
np.power(self.cov, 1)

array([[ 0.09543691, -0.06212411],
       [-0.06212411,  0.11117773]])

In [36]:
self.ev

array([[ 0.0983153 ],
       [-0.06740113]])

Ah this is it - if self.ev becomes nan it stays as nan. Need a check to prevent this

In [37]:
# Intialise VPU
vpu2 = VPU(2)
# Test Iteration
for _ in range(0, 100):
    data_in = np.random.randint(2, size=(2, 1))
    cause, residual = vpu2.iterate(data_in)
print(cause, residual)
print(vpu2.cu.covariance)
assert vpu2.cu.covariance.any()
assert cause.any()
assert residual.any()
vpu2.reset()
# assert not vpu2.cu.covariance.any()
print(vpu2.cu.covariance)


[[0.]] [[0.]
 [0.]]
[[0.23332932 0.02195817]
 [0.02195817 0.2293767 ]]


AssertionError: 

## BufferVPU

In [38]:
from src.var_processor.vpu import BufferVPU

In [39]:
vpu = BufferVPU(2, 4)
assert vpu.buffer.shape == (2, 4)
assert vpu.cu.covariance.shape == (8, 8)
assert vpu.pi.ev.shape == (8, 1)

In [40]:
reshaped = vpu.buffer.reshape(-1, 1)
print(reshaped, reshaped.shape)

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]] (8, 1)


In [41]:
# Test Iteration
for _ in range(0, 100):
    data_in = np.random.randint(2, size=(2, 1))
    cause, residual = vpu.iterate(data_in)
old_cov = vpu.cu.covariance
assert old_cov.any()
vpu.reset()
new_cov = vpu.cu.covariance
assert not np.array_equal(old_cov, new_cov)

# Sensor Object

The input into each VPU is a vector of N. Whatever the stage.

Stage.
* Internal variables (for init)
    * vec_len - vector length (N)
* Methods
    * __init__ - initialise a set of VPUs for one time stage. 
        * Input
            * stage_len - number of stages (k)
            * vec_len - vector length (N)
    * forward - input data and update VPUs.
        * Input:
            * stage_in - array of input data for stage.
        * Return:  
            * updated Rs and residuals for the stage as numpy array
    * get_cause - get the Rs from all individual VPUs. In binary form or float form?
        * Return:
            * causes - numpy array of Rs

We might actually want a "stage" object. generate_stage and process_stage are "stage" methods.

Time stages = logN(sensor_resolution)

We need a common way of getting the sensor_resolution. First time stage has sensor_resolution/N VPUs.

See:
* https://github.com/benhoyle/predictive_coding/blob/master/2019-10-28%20SpaceTime%20Grid%20object%20development.ipynb
* https://github.com/benhoyle/predictive_coding/blob/master/Time%20Filtering.ipynb

Below is very similar to our "layer" in the predictive_coding brain code. But we are flattening everything to 1D.

In [42]:
from src.var_processor.vpu import VPU

class TimeStage:
    """Object to represent a time stage of processing."""
    
    def __init__(self, vec_len, stage_len):
        """Initialise stage.
        
        Arg:
            vec_len - length of each 1D vector processed by the VPUs.
            stage_len - integer indicating number of VPUs.
        """
        self.vec_len = vec_len
        self.stage_len = stage_len
        self.size = self.vec_len*self.stage_len
        self.vpus = [VPU(vec_len) for _ in range(0, stage_len)]
        # Create a blank array for the causes
        self.causes = np.zeros(shape=(stage_len, 1))
        # Create a blank array for the residuals
        self.residuals = np.zeros(shape=(self.size, 1))
        
    def forward(self, stage_in):
        """Pass data to the stage for processing.
        
        Arg:
            stage_in - 1D numpy array with data to process.
        """
        # Create blank array to hold / pad data
        
        input_array = np.zeros(shape=(self.size, 1))
        # Check data is of right size
        if stage_in.shape[0] > self.size:
            # Crop input
            input_array = stage_in[:self.size]
        elif stage_in.shape[0] < self.size:
            input_array[:self.size] = stage_in
        else:
            input_array = stage_in
        # Iterate through VPUs, passing data in
        # Create a blank array for the causes
        causes = np.zeros
        for i, vpu in enumerate(self.vpus):
            start = i*self.vec_len
            end = (i+1)*self.vec_len
            r, residual = vpu.iterate(input_array[start:end])
            self.causes[i] = r
            self.residuals[start:end] = residual
        
    def get_causes(self):
        """Return output of VPUs as array."""
        return self.causes
    
    def get_residuals(self):
        """Return residual output as array."""
        return self.residuals
    
    def __repr__(self):
        """Print layer information."""
        string = f"There are {self.stage_len} units \n"
        string += f"with dimensionality {self.vec_len}x1"
        return string
            
        

In [43]:
stages = TimeStage(3, 10)
assert len(stages.vpus) == 10
assert not stages.causes.any()
assert not stages.residuals.any()
print(stages)

There are 10 units 
with dimensionality 3x1


In [44]:
data_in = np.random.randint(2, size=(stages.size, 1))
print(data_in.T)

stages.forward(data_in)

[[1 1 0 1 1 0 0 1 0 1 0 1 1 1 1 1 0 1 0 1 0 0 1 0 1 0 1 0 0 0]]


In [45]:
stages.causes

array([[0.7781574 ],
       [0.4959966 ],
       [0.40622868],
       [0.73711774],
       [1.70856429],
       [1.09356688],
       [0.84952028],
       [0.56916066],
       [1.18635544],
       [0.        ]])

In [46]:
stages.residuals

array([[ 0.49539255],
       [ 0.89907851],
       [-0.58370807],
       [ 0.87994584],
       [ 0.87404153],
       [-0.46447184],
       [-0.36740689],
       [ 0.83497826],
       [-0.0529315 ],
       [ 0.70340475],
       [-0.62808406],
       [ 0.75325269],
       [-0.00577201],
       [-0.15295815],
       [ 0.23953824],
       [ 0.09764776],
       [-0.54358569],
       [ 0.70646373],
       [-0.21090329],
       [ 0.2783153 ],
       [-0.39544367],
       [-0.03409935],
       [ 0.67605614],
       [-0.4667349 ],
       [ 0.1863242 ],
       [-0.62675028],
       [ 0.40623658],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ]])

We need a method to "bed in" the covariance - we need to input data for a certain number of iterations.

In [47]:
"9" in stages.__repr__()

False

SensorSource needs a get_data_size() method. Or we could add this to the Sensor methods.

Class object. 
* Internal variables (for init)
    * sensor_source - sensor that is providing the data.
    * vec_len - vector length (N)
    * time_len - length of time buffering (M)
* Methods
    * build_stages - build and initialise the time stages.
    * generate_stage - create a new time stage.
    * get_data_size - determine the 1D size of the data from the sensor source.
        * Return:
            * size - integer indicating the 1D size.
    * get_frame - get a frame of data from the sensor. Add thresholding here?
        * Return:
            * frame
    * iterate - high-level loop for all stages.
        * Return:
            * Array of Rs for visualisation?
    * cause_as_image - return the causes for each stage as an image - upscales stages with lower numbers of causes.
    * cause_pyramid - return causes as list of numpy arrays.
    * residual_pyramid - return residuals as list of numpy arrays.
    


Building the set of stages:
* Compute number of stages.
* Loop over number of stages.
    * Determine stage_len.
    * Generate stage - add returned stage to list.
* Return list of stages.


The number of stages = math.log(SR, vec_len) - 1. If SR is a power, the log is an integer value, else it is a decimal and we want to take the floor of the value and crop the data. Like previous methods we can crop right, left or centre. Or we could resize instead of cropping (more intensive but easier to implement).

```#simple image scaling to (nR x nC) size
def scale(im, nR, nC):
  nR0 = len(im)     # source number of rows 
  nC0 = len(im[0])  # source number of columns 
  return [[ im[int(nR0 * r / nR)][int(nC0 * c / nC)]  
             for c in range(nC)] for r in range(nR)]
```
Or https://docs.scipy.org/doc/numpy/reference/generated/numpy.interp.html .

We can shrink to the nearest power? This at least doesn't increase dimensionality at the cost of lossing detail. But apparently we can interpolate downwards...
For a 1D case:



In [48]:
np.linspace(0, 10-1, 10)

array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [49]:
np.linspace(0, 10-1, 5)

array([0.  , 2.25, 4.5 , 6.75, 9.  ])

In [50]:
def resize(array, elem_num):
    """Scale array.
    
    Arg:
        elem_num - integer number of new elements in array.
    """
    old_length = array.shape[0]
    x = np.linspace(0, old_length-1, elem_num)
    xp = np.linspace(0, old_length-1, old_length)
    return np.interp(x, xp, array.flatten()).reshape(-1, 1)

In [51]:
array = np.linspace(0, 10-1, 10).reshape(-1, 1)
resized = resize(array, 5)
print(resized, resized.shape)

[[0.  ]
 [2.25]
 [4.5 ]
 [6.75]
 [9.  ]] (5, 1)


In [60]:
import math


class Sensor:
    """Object to process a 1D sensor signal.

    For this to work well the data output by sensor_source should be a power
    of vec_len.
    """

    def __init__(self, sensor_source, vec_len, time_len, start=True):
        """Initialise sensor.

        Arg:
            sensor_source - SensorSource object that outputs a
            vector of sensor readings when iterated.
            vec_len - length of vector for VPU.
            time_len - length of time buffering.
        """
        self.source = sensor_source
        self.vec_len = vec_len
        self.time_len = time_len
        # Variable to store time stages
        self.stages = list()
        # Variable to store nearest power length
        self.power_len = None
        # Variable to store original sensor length
        self.sensor_len = None
        # Start sensor by default
        if start:
            self.start()

    def start(self):
        """Start sensor."""
        self.source.start()
        if not self.power_len:
            _, initial_frame = self.source.read()
            flattened = initial_frame.reshape(-1, 1)
            self.sensor_len = flattened.shape[0]
            num_stages = math.log(self.sensor_len, self.vec_len)
            self.num_stages = int(num_stages)
            self.power_len = self.vec_len**self.num_stages
        # Build the time stages
        self.build_stages()

    def get_frame(self):
        """Get a 1D frame of data from the sensor."""
        # If the sensor is not started, start
        if not self.source.started:
            self.start()
        # Get frame and flatten to 1D array
        _, initial_frame = self.source.read()
        flattened = initial_frame.reshape(-1, 1)
        # Resize to nearest power of vec_len
        output = resize(flattened, self.power_len)
        return output

    def generate_stage(self, stage_len):
        """Generate a stage.

        Arg:
            stage_len - integer number of stages.
        """
        return TimeStage(self.vec_len, stage_len)

    def build_stages(self):
        """Build a set of stages."""
        self.stages = [
            self.generate_stage(
                int(
                    self.power_len / self.vec_len**(i+1)
                )
            )
            for i in range(0, self.num_stages)
        ]

    def iterate(self):
        """High level processing loop."""
        input_data = self.get_frame()
        for stage in self.stages:
            stage.forward(input_data)
            input_data = stage.get_causes()


In [61]:
a = math.log(312, 3); a

5.227506780187165

In [62]:
int(a)

5

In [63]:
sensor = Sensor(AudioSource(), 3, 3)

In [64]:
len(sensor.stages)

10

In [65]:
data = sensor.get_frame()

In [67]:
print(data, data.shape)

[[-20842.        ]
 [-19811.53571671]
 [-20396.81892697]
 ...
 [ 14271.91840537]
 [ 14416.11538071]
 [ 15861.        ]] (59049, 1)


In [70]:
math.log(59049, 3)

9.999999999999998

In [71]:
3**10

59049

In [76]:
sensor.iterate()